In [2]:
!pip install xlrd==1.2.0

     |████████████████████████████████| 103 kB 5.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
from scipy.sparse.linalg import lsmr
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
import lightgbm as lgb
import xgboost as xgb

In [4]:
# Tricubic Kernel
def Tricubic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,70/81*(1-d**3)**3)

# Quartic Kernel
def Quartic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,15/16*(1-d**2)**2)

# Epanechnikov Kernel
def Epanechnikov(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,3/4*(1-d**2)) 

In [5]:
# Implementing new kernels
# Triangular Kernel
def Triangular(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,(1-np.abs(d))) 

# Triweight Kernel
def Triweight(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,35/32*(1-d**2)**3) 


In [6]:
#Defining the kernel local regression model

def lw_reg(X, y, xnew, kern, tau, intercept):
    n = len(X)
    yest = np.zeros(n)

    if len(y.shape)==1:
      y = y.reshape(-1,1)

    if len(X.shape)==1:
      X = X.reshape(-1,1)
    
    if intercept:
      X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
      X1 = X

    w = np.array([kern((X - X[i])/(2*tau)) for i in range(n)])

    for i in range(n):          
        W = np.diag(w[:,i])
        b = np.transpose(X1).dot(W).dot(y)
        A = np.transpose(X1).dot(W).dot(X1)

        beta, res, rnk, s = lstsq(A, b)
        yest[i] = np.dot(X1[i],beta)
    if X.shape[1]==1:
      f = interp1d(X.flatten(),yest,fill_value='extrapolate')
    else:
      f = LinearNDInterpolator(X, yest)
    output = f(xnew)
    if sum(np.isnan(output))>0:
      g = NearestNDInterpolator(X,y.ravel()) 
      output[np.isnan(output)] = g(xnew[np.isnan(output)])
    return output

In [7]:
def booster_lwrandchoice(X,y,xnew,kern,tau,model_boosting,nboost):
  Fx = lw_reg(X,y,X,kern,tau,True)
  Fx_new = lw_reg(X,y,xnew,kern,tau,True)
  new_y = y - Fx
  output = Fx
  output_new = Fx_new
  for i in range(nboost):
    model_boosting.fit(X,new_y)
    output += model_boosting.predict(X)
    output_new += model_boosting.predict(xnew)
    new_y = y - output
  return output_new

In [8]:
def booster_lwr(X,y,xnew,kern,tau,nboost):
  Fx = lw_reg(X,y,X,kern,tau,True)
  Fx_new = lw_reg(X,y,xnew,kern,tau,True)
  new_y = y - Fx
  output = Fx
  output_new = Fx_new
  for i in range(nboost):
    output += lw_reg(X,new_y,X,kern,tau,True)
    output_new += lw_reg(X,y,xnew,kern,tau,True)
    new_y = y - output
  return output_new

# Bring in Data

In [9]:
concrete = pd.read_excel("Concrete_Data.xls")

In [10]:
scale = StandardScaler()

In [11]:
# I ended up not using this version of X because it was taking an excessive amount of time to run the models when using all features
#X = concrete[['Cement (component 1)(kg in a m^3 mixture)', 'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
#              'Fly Ash (component 3)(kg in a m^3 mixture)', 'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
#              'Water  (component 4)(kg in a m^3 mixture)', 'Superplasticizer (component 5)(kg in a m^3 mixture)',
#              'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)']].values
#y = concrete["Concrete compressive strength(MPa, megapascals) "].values

In [11]:
X = concrete[['Cement (component 1)(kg in a m^3 mixture)', 'Coarse Aggregate  (component 6)(kg in a m^3 mixture)', 'Age (day)']].values
y = concrete["Concrete compressive strength(MPa, megapascals) "].values

In [30]:
model_boosting1 = RandomForestRegressor(n_estimators=5, max_depth=2)
model_boosting2 = RandomForestRegressor(n_estimators=10, max_depth=2)
model_boosting3 = RandomForestRegressor(n_estimators=5, max_depth=3)
model_boosting4 = RandomForestRegressor(n_estimators=10, max_depth=3)
model_boosting5 = RandomForestRegressor(n_estimators=50, max_depth=2)
model_boosting6 = RandomForestRegressor(n_estimators=10, max_depth=2)
model_boosting7 = RandomForestRegressor(n_estimators=50, max_depth=3)
model_boosting8 = RandomForestRegressor(n_estimators=100, max_depth=3)

In [15]:
mse_blwr_tric1 = []
mse_blwr_quart1 = []
mse_blwr_epa1 = []
mse_blwr_triw1 = []
mse_blwr_trian1 = []

mse_blwr_tric2 = []
mse_blwr_quart2 = []
mse_blwr_epa2 = []
mse_blwr_triw2 = []
mse_blwr_trian2 = []

mse_blwr_tric3 = []
mse_blwr_quart3 = []
mse_blwr_epa3 = []
mse_blwr_triw3 = []
mse_blwr_trian3 = []

mse_blwr_tric4 = []
mse_blwr_quart4 = []
mse_blwr_epa4 = []
mse_blwr_triw4 = []
mse_blwr_trian4 = []

mse_blwr_tric5 = []
mse_blwr_quart5 = []
mse_blwr_epa5 = []
mse_blwr_triw5 = []
mse_blwr_trian5 = []

mse_blwr_tric6 = []
mse_blwr_quart6 = []
mse_blwr_epa6 = []
mse_blwr_triw6 = []
mse_blwr_trian6 = []

mse_blwr_tric7 = []
mse_blwr_quart7 = []
mse_blwr_epa7 = []
mse_blwr_triw7 = []
mse_blwr_trian7 = []

mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []


kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting1,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting1,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting1,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting1,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting1,2)

  mse_blwr_tric1.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart1.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa1.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw1.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian1.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting2,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting2,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting2,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting2,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting2,2)

  mse_blwr_tric2.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart2.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa2.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw2.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian2.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting3,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting3,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting3,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting3,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting3,2)

  mse_blwr_tric3.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart3.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa3.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw3.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian3.append(mse(ytest, yhat_blwr_5))


  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting4,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting4,2)

  mse_blwr_tric4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa4.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw4.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian4.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting5,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting5,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting5,2)

  mse_blwr_tric5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa5.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw5.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian5.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting6,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting6,2)

  mse_blwr_tric6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa6.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw6.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian6.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting7,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting7,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting7,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting7,2)

  mse_blwr_tric7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa7.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw7.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian7.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian1)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : '+str(np.mean(mse_blwr_tric2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian2)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian3)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 93.04766680880306
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 90.87161503816124
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 92.11625876986315
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 91.9402835487554
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 93.36078552187332


The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : 90.42685281074014
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 89.92265997375904
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 92.40421095707234
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 90.0886269950209
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 90.39621160869156


The Cross-validated Mean Squared Error for Boosted LWR with tricu

Best was triweight at a MSE of 84.955 and 84.900

In [31]:
mse_blwr_triw7 = []
mse_blwr_quart7 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting7,2)

  mse_blwr_triw7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.95173847250389
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.93457910056665


The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : 84.93350355078776
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.93608255071352


Best kernel was quartic with an MSE of 83.24 and 83.32

In [39]:
mse_blwr_triw7 = []
mse_blwr_quart7 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
  
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting7,2)

  mse_blwr_triw7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.41424846563429
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.37699395804587


The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : 83.28758961088658
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.20880790353138


Best was quartic with an MSE of 82.66 and 82.72

In [38]:
mse_blwr_triw7 = []
mse_blwr_quart7 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
  
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting7,2)

  mse_blwr_triw7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 82.66276011757859
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 82.71253899193417


The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : 82.7226507690194
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 82.68421331359411


Best was quartic with an MSE of 77.62 and 77.68

In [37]:
mse_blwr_triw7 = []
mse_blwr_quart7 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
  
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting7,2)

  mse_blwr_triw7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 76.86308793312824
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 77.84271286529187


The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : 77.02538144179594
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 77.67995818771124


Best was quartic with an MSE of 79.448 and 79.788

In [36]:
mse_blwr_triw7 = []
mse_blwr_quart7 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
  
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting7,2)

  mse_blwr_triw7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 78.57291577140222
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 79.49339210232168


The Cross-validated Mean Squared Error for Boosted LWR with triweight is  : 79.600176643588
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 80.36275757776538


Alternate method - did not work as well

In [25]:
mse_blwr_tric1 = []
mse_blwr_quart1 = []
mse_blwr_epa1 = []
mse_blwr_triw1 = []
mse_blwr_trian1 = []

mse_blwr_tric2 = []
mse_blwr_quart2 = []
mse_blwr_epa2 = []
mse_blwr_triw2 = []
mse_blwr_trian2 = []

mse_blwr_tric3 = []
mse_blwr_quart3 = []
mse_blwr_epa3 = []
mse_blwr_triw3 = []
mse_blwr_trian3 = []

mse_blwr_tric4 = []
mse_blwr_quart4 = []
mse_blwr_epa4 = []
mse_blwr_triw4 = []
mse_blwr_trian4 = []

mse_blwr_tric5 = []
mse_blwr_quart5 = []
mse_blwr_epa5 = []
mse_blwr_triw5 = []
mse_blwr_trian5 = []

mse_blwr_tric6 = []
mse_blwr_quart6 = []
mse_blwr_epa6 = []
mse_blwr_triw6 = []
mse_blwr_trian6 = []

mse_blwr_tric7 = []
mse_blwr_quart7 = []
mse_blwr_epa7 = []
mse_blwr_triw7 = []
mse_blwr_trian7 = []

mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []


kf = KFold(n_splits=3,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric1.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart1.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa1.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw1.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian1.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric2.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart2.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa2.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw2.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian2.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric3.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart3.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa3.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw3.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian3.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa4.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw4.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian4.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa5.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw5.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian5.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa6.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw6.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian6.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa7.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw7.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian7.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwr(xtrain,ytrain,xtest, Tricubic,1,2)
  yhat_blwr_2 = booster_lwr(xtrain,ytrain,xtest, Quartic,1,2)
  yhat_blwr_3 = booster_lwr(xtrain,ytrain,xtest, Epanechnikov,1,2)
  yhat_blwr_4 = booster_lwr(xtrain,ytrain,xtest, Triweight,1,2)
  yhat_blwr_5 = booster_lwr(xtrain,ytrain,xtest, Triangular,1,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))


print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian1)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : '+str(np.mean(mse_blwr_tric2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian2)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian3)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 5247.268711964701
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 5278.916059196176
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 5273.533214448678
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 5279.397341317413
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 5300.627659972757


The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : 5247.268711964701
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 5278.916059196176
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 5273.533214448678
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 5279.397341317413
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 5300.627659972757


The Cross-validated Mean Squared Error for Boosted LWR with tri

In [12]:
model_boosting1 = xgb.XGBRegressor(max_depth=2, n_estimators=10, objective ='reg:squarederror')
model_boosting2 = xgb.XGBRegressor(max_depth=3, n_estimators=10, objective ='reg:squarederror')
model_boosting3 = xgb.XGBRegressor(max_depth=2, n_estimators=50, objective ='reg:squarederror')
model_boosting4 = xgb.XGBRegressor(max_depth=3, n_estimators=50, objective ='reg:squarederror')
model_boosting5 = xgb.XGBRegressor(max_depth=2, n_estimators=100, objective ='reg:squarederror')
model_boosting6 = xgb.XGBRegressor(max_depth=3, n_estimators=100, objective ='reg:squarederror')

In [17]:
mse_blwr_tric1 = []
mse_blwr_quart1 = []
mse_blwr_epa1 = []
mse_blwr_triw1 = []
mse_blwr_trian1 = []

mse_blwr_tric2 = []
mse_blwr_quart2 = []
mse_blwr_epa2 = []
mse_blwr_triw2 = []
mse_blwr_trian2 = []

mse_blwr_tric3 = []
mse_blwr_quart3 = []
mse_blwr_epa3 = []
mse_blwr_triw3 = []
mse_blwr_trian3 = []

mse_blwr_tric4 = []
mse_blwr_quart4 = []
mse_blwr_epa4 = []
mse_blwr_triw4 = []
mse_blwr_trian4 = []

mse_blwr_tric5 = []
mse_blwr_quart5 = []
mse_blwr_epa5 = []
mse_blwr_triw5 = []
mse_blwr_trian5 = []

mse_blwr_tric6 = []
mse_blwr_quart6 = []
mse_blwr_epa6 = []
mse_blwr_triw6 = []
mse_blwr_trian6 = []


kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting1,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting1,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting1,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting1,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting1,2)

  mse_blwr_tric1.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart1.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa1.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw1.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian1.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting2,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting2,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting2,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting2,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting2,2)

  mse_blwr_tric2.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart2.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa2.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw2.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian2.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting3,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting3,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting3,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting3,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting3,2)

  mse_blwr_tric3.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart3.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa3.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw3.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian3.append(mse(ytest, yhat_blwr_5))


  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting4,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting4,2)

  mse_blwr_tric4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa4.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw4.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian4.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting5,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting5,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting5,2)

  mse_blwr_tric5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa5.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw5.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian5.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting6,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting6,2)

  mse_blwr_tric6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa6.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw6.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian6.append(mse(ytest, yhat_blwr_5))


print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian1)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : '+str(np.mean(mse_blwr_tric2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian2)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian3)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian6)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 94.02176847660436
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 94.59113664865455
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 96.05760088385459
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 92.92509959843306
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 94.6478275949769


The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : 86.95656119371242
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 86.64716715099696
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 88.11190122561672
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.5817701130974
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 86.88119170940479


The Cross-validated Mean Squared Error for Boosted LWR with tricu

Triweight was best with MSE of 84.978, 84.965, and 84.98

In [13]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []
mse_blwr_tric4 = []

mse_blwr_triw5 = []
mse_blwr_quart5 = []
mse_blwr_tric5 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []
mse_blwr_tric6 = []




kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.01,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric4.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.01,model_boosting5,2)

  mse_blwr_triw5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric5.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.01,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric6.append(mse(ytest, yhat_blwr_3))

print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))

The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.98165448528218
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.97766514708044
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 84.98260076817579


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.9668595431247
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.9650501622164
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 84.96746344260485


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.98494390208647
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.98077310535021
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 84.98542127990825


Quartic with an MSE of 83.31, 83.368, and 83.48

In [14]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []
mse_blwr_tric4 = []

mse_blwr_triw5 = []
mse_blwr_quart5 = []
mse_blwr_tric5 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []
mse_blwr_tric6 = []




kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.05,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric4.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.05,model_boosting5,2)

  mse_blwr_triw5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric5.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.05,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric6.append(mse(ytest, yhat_blwr_3))

print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))

The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.30840336061506
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.35491042167773
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 83.4274754483645


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.36816743617615
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.37269985589559
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 83.36966308950663


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.48070911552598
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.54095848841271
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 83.53605013189394


Best was quartic with an MSE of 81.07 and 81.44, and 80.22

In [15]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []
mse_blwr_tric4 = []

mse_blwr_triw5 = []
mse_blwr_quart5 = []
mse_blwr_tric5 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []
mse_blwr_tric6 = []




kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.1,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric4.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.1,model_boosting5,2)

  mse_blwr_triw5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric5.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.1,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric6.append(mse(ytest, yhat_blwr_3))

print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))

The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 81.0683128369599
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 81.63725944415367
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 81.53921736766995


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 81.43830549960397
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 81.6599448733882
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 81.7270329472694


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 80.2180163688902
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 80.94461999875037
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 80.44494435721751


Best was quartic at an MSE of 63.73, 64.695; tricubic with an MSE of 56.72

In [16]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []
mse_blwr_tric4 = []

mse_blwr_triw5 = []
mse_blwr_quart5 = []
mse_blwr_tric5 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []
mse_blwr_tric6 = []




kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.5,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric4.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.5,model_boosting5,2)

  mse_blwr_triw5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric5.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.5,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric6.append(mse(ytest, yhat_blwr_3))

print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))

The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 63.72701803030511
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 64.40411730676756
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 64.26803524781606


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 64.6952706622424
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 64.75321869073635
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 64.8324708115349


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 56.83412105692558
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 57.26189821342807
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 56.71624649333512


Best was triweight with an MSE of 64.21, 64.35; quartic with an MSE of 55.25

In [17]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []
mse_blwr_tric4 = []

mse_blwr_triw5 = []
mse_blwr_quart5 = []
mse_blwr_tric5 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []
mse_blwr_tric6 = []




kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric4.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting5,2)

  mse_blwr_triw5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric5.append(mse(ytest, yhat_blwr_3))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_tric6.append(mse(ytest, yhat_blwr_3))

print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))

The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 64.25465224410843
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 64.2096107894063
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 64.87382360487751


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 64.76403805729767
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 64.35099986718724
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 64.63601891468674


The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 55.24796965356792
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 55.286138335966925
The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 55.607131628511375


In [24]:
model_boosting1 = lgb.LGBMRegressor(learning_rate=0.05, n_estimators=5, max_depth=2)
model_boosting2 = lgb.LGBMRegressor(learning_rate=0.05, n_estimators=5, max_depth=3)
model_boosting3 = lgb.LGBMRegressor(learning_rate=0.05, n_estimators=10, max_depth=2)
model_boosting4 = lgb.LGBMRegressor(learning_rate=0.05, n_estimators=10, max_depth=3)
model_boosting5 = lgb.LGBMRegressor(learning_rate=0.1, n_estimators=5, max_depth=2)
model_boosting6 = lgb.LGBMRegressor(learning_rate=0.1, n_estimators=5, max_depth=3)
model_boosting7 = lgb.LGBMRegressor(learning_rate=0.1, n_estimators=10, max_depth=2)
model_boosting8 = lgb.LGBMRegressor(learning_rate=0.1, n_estimators=10, max_depth=3)

In [19]:
mse_blwr_tric1 = []
mse_blwr_quart1 = []
mse_blwr_epa1 = []
mse_blwr_triw1 = []
mse_blwr_trian1 = []

mse_blwr_tric2 = []
mse_blwr_quart2 = []
mse_blwr_epa2 = []
mse_blwr_triw2 = []
mse_blwr_trian2 = []

mse_blwr_tric3 = []
mse_blwr_quart3 = []
mse_blwr_epa3 = []
mse_blwr_triw3 = []
mse_blwr_trian3 = []

mse_blwr_tric4 = []
mse_blwr_quart4 = []
mse_blwr_epa4 = []
mse_blwr_triw4 = []
mse_blwr_trian4 = []

mse_blwr_tric5 = []
mse_blwr_quart5 = []
mse_blwr_epa5 = []
mse_blwr_triw5 = []
mse_blwr_trian5 = []

mse_blwr_tric6 = []
mse_blwr_quart6 = []
mse_blwr_epa6 = []
mse_blwr_triw6 = []
mse_blwr_trian6 = []

mse_blwr_tric7 = []
mse_blwr_quart7 = []
mse_blwr_epa7 = []
mse_blwr_triw7 = []
mse_blwr_trian7 = []

mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []


kf = KFold(n_splits=3,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting1,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting1,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting1,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting1,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting1,2)

  mse_blwr_tric1.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart1.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa1.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw1.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian1.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting2,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting2,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting2,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting2,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting2,2)

  mse_blwr_tric2.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart2.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa2.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw2.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian2.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting3,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting3,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting3,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting3,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting3,2)

  mse_blwr_tric3.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart3.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa3.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw3.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian3.append(mse(ytest, yhat_blwr_5))


  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting4,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting4,2)

  mse_blwr_tric4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa4.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw4.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian4.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting5,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting5,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting5,2)

  mse_blwr_tric5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa5.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw5.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian5.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting6,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting6,2)

  mse_blwr_tric6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa6.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw6.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian6.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting7,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting7,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting7,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting7,2)

  mse_blwr_tric7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa7.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw7.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian7.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian1)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : '+str(np.mean(mse_blwr_tric2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian2)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian3)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 106.44791466700211
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 106.12328892003579
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 108.42175559632562
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 104.2849885128415
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 106.64272654114086


The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : 102.3348288696688
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 101.9870758221126
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 104.08646380676805
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 100.6306077341993
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 102.61747099568971


The Cross-validated Mean Squared Error for Boosted LWR wi

Best was quartic with MSE of 84.959 for all three models

In [25]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted with triweight LWR is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted with quartic LWR is : '+str(np.mean(mse_blwr_quart6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR quartic is : '+str(np.mean(mse_blwr_quart6)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.96027968498272
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.95867306224493


The Cross-validated Mean Squared Error for Boosted with triweight LWR is : 84.96050647301347
The Cross-validated Mean Squared Error for Boosted with quartic LWR is : 84.95895917412864


The Cross-validated Mean Squared Error for Boosted LWR triweight is : 84.96050647301347
The Cross-validated Mean Squared Error for Boosted LWR quartic is : 84.95895917412864


Best was quartic with an MSE of 83.519 and 83.520 for the final two models

In [26]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted with triweight LWR is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted with quartic LWR is : '+str(np.mean(mse_blwr_quart6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR quartic is : '+str(np.mean(mse_blwr_quart6)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.56412454044627
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.51939095075721


The Cross-validated Mean Squared Error for Boosted with triweight LWR is : 83.56640440313761
The Cross-validated Mean Squared Error for Boosted with quartic LWR is : 83.5200060002496


The Cross-validated Mean Squared Error for Boosted LWR triweight is : 83.56640440313761
The Cross-validated Mean Squared Error for Boosted LWR quartic is : 83.5200060002496


Best kernel was triweight at an MSE of 83.358 and of 83.3546 for the remaining two

In [27]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted with triweight LWR is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted with quartic LWR is : '+str(np.mean(mse_blwr_quart6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR quartic is : '+str(np.mean(mse_blwr_quart6)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.35824272250802
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.46123664613545


The Cross-validated Mean Squared Error for Boosted with triweight LWR is : 83.35457352123584
The Cross-validated Mean Squared Error for Boosted with quartic LWR is : 83.46572264240095


The Cross-validated Mean Squared Error for Boosted LWR triweight is : 83.35457352123584
The Cross-validated Mean Squared Error for Boosted LWR quartic is : 83.46572264240095


Best was triweight at an MSE of 84.327 and approximately 84.52888

In [28]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted with triweight LWR is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted with quartic LWR is : '+str(np.mean(mse_blwr_quart6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR quartic is : '+str(np.mean(mse_blwr_quart6)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.32747711162502
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 85.765446980415


The Cross-validated Mean Squared Error for Boosted with triweight LWR is : 84.52888488714599
The Cross-validated Mean Squared Error for Boosted with quartic LWR is : 85.38835454726046


The Cross-validated Mean Squared Error for Boosted LWR triweight is : 84.52888488714599
The Cross-validated Mean Squared Error for Boosted LWR quartic is : 85.38835454726046


Best was triweight with an MSE of 91.228 and 91.3485

In [29]:
mse_blwr_triw4 = []
mse_blwr_quart4 = []


mse_blwr_triw6 = []
mse_blwr_quart6 = []


mse_blwr_triw8 = []
mse_blwr_quart8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)

  mse_blwr_triw4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)

  mse_blwr_triw6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)

  mse_blwr_triw8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))

print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted with triweight LWR is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted with quartic LWR is : '+str(np.mean(mse_blwr_quart6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR quartic is : '+str(np.mean(mse_blwr_quart6)))

The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 91.22787725748537
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 92.66635230745787


The Cross-validated Mean Squared Error for Boosted with triweight LWR is : 91.34850201174677
The Cross-validated Mean Squared Error for Boosted with quartic LWR is : 92.69693984356529


The Cross-validated Mean Squared Error for Boosted LWR triweight is : 91.34850201174677
The Cross-validated Mean Squared Error for Boosted LWR quartic is : 92.69693984356529


In [20]:
model_boosting1 = xgb.XGBRegressor(max_depth=2, learning_rate=0.05, n_estimators=5, objective='reg:squarederror')
model_boosting2 = xgb.XGBRegressor(max_depth=3, learning_rate=0.05, n_estimators=5, objective='reg:squarederror')
model_boosting3 = xgb.XGBRegressor(max_depth=2, learning_rate=0.05, n_estimators=10, objective='reg:squarederror')
model_boosting4 = xgb.XGBRegressor(max_depth=3, learning_rate=0.05, n_estimators=10, objective='reg:squarederror')
model_boosting5 = xgb.XGBRegressor(max_depth=2, learning_rate=0.1, n_estimators=5, objective='reg:squarederror')
model_boosting6 = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=5, objective='reg:squarederror')
model_boosting7 = xgb.XGBRegressor(max_depth=2, learning_rate=0.1, n_estimators=10, objective='reg:squarederror')
model_boosting8 = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=10, objective='reg:squarederror')

In [21]:
mse_blwr_tric1 = []
mse_blwr_quart1 = []
mse_blwr_epa1 = []
mse_blwr_triw1 = []
mse_blwr_trian1 = []

mse_blwr_tric2 = []
mse_blwr_quart2 = []
mse_blwr_epa2 = []
mse_blwr_triw2 = []
mse_blwr_trian2 = []

mse_blwr_tric3 = []
mse_blwr_quart3 = []
mse_blwr_epa3 = []
mse_blwr_triw3 = []
mse_blwr_trian3 = []

mse_blwr_tric4 = []
mse_blwr_quart4 = []
mse_blwr_epa4 = []
mse_blwr_triw4 = []
mse_blwr_trian4 = []

mse_blwr_tric5 = []
mse_blwr_quart5 = []
mse_blwr_epa5 = []
mse_blwr_triw5 = []
mse_blwr_trian5 = []

mse_blwr_tric6 = []
mse_blwr_quart6 = []
mse_blwr_epa6 = []
mse_blwr_triw6 = []
mse_blwr_trian6 = []

mse_blwr_tric7 = []
mse_blwr_quart7 = []
mse_blwr_epa7 = []
mse_blwr_triw7 = []
mse_blwr_trian7 = []

mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []


kf = KFold(n_splits=3,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting1,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting1,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting1,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting1,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting1,2)

  mse_blwr_tric1.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart1.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa1.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw1.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian1.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting2,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting2,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting2,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting2,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting2,2)

  mse_blwr_tric2.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart2.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa2.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw2.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian2.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting3,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting3,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting3,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting3,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting3,2)

  mse_blwr_tric3.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart3.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa3.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw3.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian3.append(mse(ytest, yhat_blwr_5))


  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting4,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting4,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting4,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting4,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting4,2)

  mse_blwr_tric4.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart4.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa4.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw4.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian4.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting5,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting5,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting5,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting5,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting5,2)

  mse_blwr_tric5.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart5.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa5.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw5.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian5.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting6,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting6,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting6,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting6,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting6,2)

  mse_blwr_tric6.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart6.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa6.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw6.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian6.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting7,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting7,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting7,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting7,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting7,2)

  mse_blwr_tric7.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart7.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa7.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw7.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian7.append(mse(ytest, yhat_blwr_5))

  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))


print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw1)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian1)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : '+str(np.mean(mse_blwr_tric2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw2)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian2)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw3)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian3)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw4)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian4)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw5)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian5)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw6)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian6)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw7)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian7)))
print('\n')
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 106.6315730023709
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 106.26337074804047
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 108.6222750057576
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 104.42273586588006
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 106.87429288316714


The Cross-validated Mean Squared Error for Boosted LWR with tricubic is  : 103.20497510436348
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 102.85538364792178
The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : 104.57005277791374
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 101.28485482933468
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 103.00849547776356


The Cross-validated Mean Squared Error for Boosted LWR 

Best kernel is Epanechnikov with an MSE of 84.8331

In [22]:
mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.01,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.01,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,0.01,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.01,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,0.01,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

 
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 84.8334589025137
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 84.83351118973668
The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : 84.83314534806277
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.8337738736759
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 84.83339321065894


Best kernel is quartic with MSE of 83.021

In [23]:
mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.05,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.05,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,0.05,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.05,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,0.05,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

 
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 83.02256292095133
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 83.02127755966087
The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : 83.11864718020949
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 83.07337748316098
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 83.11730526066853


Best kernel is triweight with MSE of 82.697

In [24]:
mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.1,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,0.1,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.1,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,0.1,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

 
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 83.05931522533888
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 82.82787840603324
The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : 83.35191634421291
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 82.69662666267686
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 82.87991663373369


Best kernel was triweight with MSE of 79.94

In [25]:
mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,0.5,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,0.5,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,0.5,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,0.5,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,0.5,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

 
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 80.56406169659894
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 80.95065720266257
The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : 81.53377699908796
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 79.93539944148537
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 81.04833617728374


Best kernel was triweight with an MSE of 84.58

In [26]:
mse_blwr_tric8 = []
mse_blwr_quart8 = []
mse_blwr_epa8 = []
mse_blwr_triw8 = []
mse_blwr_trian8 = []



kf = KFold(n_splits=5,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)
    
  yhat_blwr_1 = booster_lwrandchoice(xtrain,ytrain,xtest, Tricubic,1,model_boosting8,2)
  yhat_blwr_2 = booster_lwrandchoice(xtrain,ytrain,xtest, Quartic,1,model_boosting8,2)
  yhat_blwr_3 = booster_lwrandchoice(xtrain,ytrain,xtest, Epanechnikov,1,model_boosting8,2)
  yhat_blwr_4 = booster_lwrandchoice(xtrain,ytrain,xtest, Triweight,1,model_boosting8,2)
  yhat_blwr_5 = booster_lwrandchoice(xtrain,ytrain,xtest, Triangular,1,model_boosting8,2)

  mse_blwr_tric8.append(mse(ytest, yhat_blwr_1))
  mse_blwr_quart8.append(mse(ytest, yhat_blwr_2))
  mse_blwr_epa8.append(mse(ytest, yhat_blwr_3))
  mse_blwr_triw8.append(mse(ytest, yhat_blwr_4))
  mse_blwr_trian8.append(mse(ytest, yhat_blwr_5))

 
print('The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : '+str(np.mean(mse_blwr_tric8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with quartic is : '+str(np.mean(mse_blwr_quart8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with Epanechnikov is : '+str(np.mean(mse_blwr_epa8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triweight is : '+str(np.mean(mse_blwr_triw8)))
print('The Cross-validated Mean Squared Error for Boosted LWR with triangular is : '+str(np.mean(mse_blwr_trian8)))

The Cross-validated Mean Squared Error for Boosted LWR with tricubic is : 86.95656119371242
The Cross-validated Mean Squared Error for Boosted LWR with quartic is : 86.64716715099696
The Cross-validated Mean Squared Error for Boosted LWR with Epanechikov is : 88.11190122561672
The Cross-validated Mean Squared Error for Boosted LWR with triweight is : 84.5817701130974
The Cross-validated Mean Squared Error for Boosted LWR with triangular is : 86.88119170940479


-----

-----

----

-------------------------